In [1]:
import pandas as pd     
from SmilesPE.pretokenizer import atomwise_tokenizer
import pdb      #pdb.set_trace()
import pubchempy as pcp

from IPython.display import Audio
#sound_file = '../applause.mp3'
sound_file = '../diving.mp3'

## Lenselink

In [2]:
lense = pd.read_table('../datasets/Lenselink_et_al/Lenselink_sample_42.txt', usecols=['CMP_CHEMBL_ID'])
#lense['CMP_CHEMBL_ID'] = lense['CMP_CHEMBL_ID'].astype(object)

#### Mapping to get SMILES created by PubChem

In [3]:
unichem = pd.read_table('../datasets/UniChem_Mapping/src1src22.txt')
unichem.rename(columns={"From src:'1'": "chembl_id", "To src:'22'": "pubchem_id"}, inplace=True)
unichem = unichem.astype(dtype={'pubchem_id':'object'})

In [4]:
lense = pd.merge(lense, unichem,
                  how='left',
                  left_on='CMP_CHEMBL_ID', right_on='chembl_id')
lense.drop_duplicates(subset='pubchem_id', keep='first', inplace=True, ignore_index=True)
lense.dropna(axis=0, how='any', inplace=True)
lense.reset_index(drop=True)

,CMP_CHEMBL_ID,chembl_id,pubchem_id
0,CHEMBL1830131,CHEMBL1830131,677915
1,CHEMBL2325093,CHEMBL2325093,2725645
2,CHEMBL2333999,CHEMBL2333999,19077515
3,CHEMBL2335013,CHEMBL2335013,71525248
4,CHEMBL2336359,CHEMBL2336359,787183
...,...,...,...
9667,CHEMBL2063917,CHEMBL2063917,66552363
9668,CHEMBL2069482,CHEMBL2069482,67497640
9669,CHEMBL2058075,CHEMBL2058075,9512200
9670,CHEMBL2172052,CHEMBL2172052,71453563


Now I use PubChemPy to look up the canonical SMILES for that PubChemID.

In [5]:
lense['canonical_smiles'] = ''
for row in lense.index:
    entry = pcp.Compound.from_cid(lense.iat[row, 2])
    lense.iat[row, 3] = entry.canonical_smiles
    
#Audio(sound_file, autoplay=True) # to notify me when it's done (this cell takes several minutes to run)

KeyboardInterrupt: 

In [14]:
lense.iloc[9170:9180]

,CMP_CHEMBL_ID,chembl_id,pubchem_id,canonical_smiles
9171,CHEMBL248853,CHEMBL248853,23585148,CCN1C2=C(C=C1C3=NC(=NC=C3)N)C(=O)NCC2
9172,CHEMBL442432,CHEMBL442432,44447430,CC1=CC=CC=C1C2CCN(CC2)S(=O)(=O)CC3(CCN(CC3)C(=...
9173,CHEMBL442291,CHEMBL442291,11504653,CC1=C(C=CC(=C1)F)C2CCN(CC2)S(=O)(=O)CC3(CCN(CC...
9174,CHEMBL254492,CHEMBL254492,11598899,CC1=CC=CC=C1C2CCN(CC2)S(=O)(=O)CC3(CCN(CC3)C(=...
9175,CHEMBL253652,CHEMBL253652,11577517,CC1=C(C=CC(=C1)C#N)C2=CCN(CC2)S(=O)(=O)CC3(CCN...
9176,CHEMBL523173,CHEMBL523173,44573402,CC(C)C1=CC(=C(C=C1OC2=CN=C(N=C2N)NC(C)CO)I)OC
9177,CHEMBL598471,CHEMBL598471,46231494,CC(CN1CCN(CC1)C2=NC=CC=N2)NC(=O)C3=CC(=NC(=C3)...
9178,CHEMBL596814,CHEMBL596814,46231351,CC(CN1CCN(CC1)C2=NC=CC=N2)NC(=O)C3=CC(=NN3C4=C...
9179,CHEMBL598497,CHEMBL598497,16069006,
9180,CHEMBL467624,CHEMBL467624,24971360,


In [15]:
#lense.to_csv('tmp_lense_2020_07_31.csv', index=False)

ATTENTION: I don't understand the error from the cell above, but the code works nonethless (I checked with the PubChem Website).

In [51]:
lense['tokens'] = ''
for row in lense.index:
    lense.at[row, 'tokens'] = atomwise_tokenizer(lense.at[row, 'canonical_smiles'])

In [53]:
alphabet_lense = set()

for i in lense.tokens:
    alphabet_lense.update(i)

In [54]:
print(alphabet_lense)

{'2', '7', '3', 'C', '[O-]', '[PH+]', 'N', '[N+]', 'Br', '6', '#', '8', '[S+]', 'Cl', '=', '(', '[N-]', 'O', '1', '5', 'S', '[NH+]', 'I', '[NH3+]', '9', ')', '4', 'P', 'F'}


## PubChem

In [57]:
pubchem = pd.read_table('../datasets/PubChem/PubChem_sample_1.txt', header=None, usecols=[1])

In [61]:
pubchem

,smiles
0,C[NH+](C)CC[NH2+]C(C1=CC=CC=C1)C2=CC=C(C=C2)Cl...
1,C=C=CC(Cl)(Cl)Cl
2,CCC1=CC(=O)C2=C(O1)C(=C(C=C2)O)C[NH+](C)C.[Cl-]
3,C1=CC(=NCl)C(=CC1=NCl)Cl
4,C(C1=C(C(=C(C(=C1Cl)Cl)Cl)CN)Cl)N
...,...
9995,C1=CC=C2C(=C1)C=CC=C2C3=NC(=NC(=N3)C4=CC=C(C=C...
9996,CC1CCC=C(N1)C2=CC=C(CC2C3CC=C4C(=C3)C=CC5=CC=C...
9997,CCCC1=C(C=CC(=C1)C)C2=NOC(=O)N2
9998,CC(C)C1CC2CC(CC2C1)SC


In [59]:
pubchem.dropna(how='any', inplace=True)
pubchem.reset_index(drop=True)

,1
0,C[NH+](C)CC[NH2+]C(C1=CC=CC=C1)C2=CC=C(C=C2)Cl...
1,C=C=CC(Cl)(Cl)Cl
2,CCC1=CC(=O)C2=C(O1)C(=C(C=C2)O)C[NH+](C)C.[Cl-]
3,C1=CC(=NCl)C(=CC1=NCl)Cl
4,C(C1=C(C(=C(C(=C1Cl)Cl)Cl)CN)Cl)N
...,...
9995,C1=CC=C2C(=C1)C=CC=C2C3=NC(=NC(=N3)C4=CC=C(C=C...
9996,CC1CCC=C(N1)C2=CC=C(CC2C3CC=C4C(=C3)C=CC5=CC=C...
9997,CCCC1=C(C=CC(=C1)C)C2=NOC(=O)N2
9998,CC(C)C1CC2CC(CC2C1)SC


In [60]:
pubchem.rename(columns={1:'smiles'}, inplace=True)

In [62]:
alphabet_pubchem = set()

for row in pubchem.index:
    tokens = atomwise_tokenizer(pubchem.iloc[row,0])
    alphabet_pubchem.update(tokens)

In [63]:
alphabet_pubchem

{'#',
 '%10',
 '%11',
 '%12',
 '%13',
 '(',
 ')',
 '.',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '=',
 'B',
 'Br',
 'C',
 'Cl',
 'F',
 'I',
 'N',
 'O',
 'P',
 'S',
 '[11CH2]',
 '[11CH3]',
 '[11CH]',
 '[13CH3]',
 '[13CH]',
 '[13C]',
 '[16OH]',
 '[2H]',
 '[Ac]',
 '[Ag]',
 '[Al]',
 '[As+3]',
 '[As]',
 '[Au+]',
 '[Au]',
 '[B-]',
 '[B]',
 '[Ba+2]',
 '[Br-]',
 '[C+]',
 '[C-]',
 '[C@@H]',
 '[C@@]',
 '[C@H]',
 '[C@]',
 '[CH-]',
 '[CH2-]',
 '[CH2]',
 '[CH3-]',
 '[CH]',
 '[C]',
 '[Cl-]',
 '[Co+3]',
 '[Co]',
 '[Cr+3]',
 '[Cr]',
 '[Cs+]',
 '[Cu+2]',
 '[Cu+]',
 '[Er+3]',
 '[Fe+2]',
 '[Ga]',
 '[Gd+3]',
 '[Ge]',
 '[H+]',
 '[HH]',
 '[Hf]',
 '[Ho]',
 '[I-]',
 '[Ir+3]',
 '[Ir]',
 '[K+]',
 '[Li+]',
 '[Mg+2]',
 '[Mg]',
 '[Mn]',
 '[Mo+4]',
 '[Mo]',
 '[N+]',
 '[N-]',
 '[N@@+]',
 '[NH+]',
 '[NH-]',
 '[NH2+]',
 '[NH2-]',
 '[NH3+]',
 '[NH4+]',
 '[N]',
 '[Na+]',
 '[Na]',
 '[Nd]',
 '[Ni+2]',
 '[Ni]',
 '[O+]',
 '[O-]',
 '[OH+]',
 '[OH-]',
 '[OH2+]',
 '[P+]',
 '[P-]',
 '[P@@]',
 '[P@]',
 '[Pb+2

## Compare the alphabets

In [64]:
alphabet_lense.difference(alphabet_pubchem) # die Elemente, die nicht in PubChem sind

{'[PH+]'}

In [81]:
print(sorted(alphabet_pubchem.difference(alphabet_lense))) # die Elemente, die nicht in Lense sind

['%10', '%11', '%12', '%13', '.', '/', 'B', '[11CH2]', '[11CH3]', '[11CH]', '[13CH3]', '[13CH]', '[13C]', '[16OH]', '[2H]', '[Ac]', '[Ag]', '[Al]', '[As+3]', '[As]', '[Au+]', '[Au]', '[B-]', '[B]', '[Ba+2]', '[Br-]', '[C+]', '[C-]', '[C@@H]', '[C@@]', '[C@H]', '[C@]', '[CH-]', '[CH2-]', '[CH2]', '[CH3-]', '[CH]', '[C]', '[Cl-]', '[Co+3]', '[Co]', '[Cr+3]', '[Cr]', '[Cs+]', '[Cu+2]', '[Cu+]', '[Er+3]', '[Fe+2]', '[Ga]', '[Gd+3]', '[Ge]', '[H+]', '[HH]', '[Hf]', '[Ho]', '[I-]', '[Ir+3]', '[Ir]', '[K+]', '[Li+]', '[Mg+2]', '[Mg]', '[Mn]', '[Mo+4]', '[Mo]', '[N@@+]', '[NH-]', '[NH2+]', '[NH2-]', '[NH4+]', '[N]', '[Na+]', '[Na]', '[Nd]', '[Ni+2]', '[Ni]', '[O+]', '[OH+]', '[OH-]', '[OH2+]', '[P+]', '[P-]', '[P@@]', '[P@]', '[Pb+2]', '[Pd+2]', '[Pd]', '[Po]', '[Pr]', '[Pt+2]', '[Rh+2]', '[Rh]', '[Ru+]', '[Ru]', '[S-]', '[S@@]', '[S@]', '[SH+]', '[Sc+3]', '[Se]', '[Si@@]', '[SiH]', '[Si]', '[Sn]', '[Te]', '[Ti+2]', '[Ti+4]', '[Ti+]', '[Ti]', '[U+2]', '[V+2]', '[W+2]', '[W]', '[Y]', '[Zn+2]', 

In [68]:
print(alphabet_lense.intersection(alphabet_pubchem))

{'2', '7', '3', 'C', '[O-]', 'N', '[N+]', 'Br', '6', '#', '8', '[S+]', 'Cl', '=', '(', '[N-]', 'O', '1', '5', 'S', '[NH+]', 'I', '[NH3+]', '9', ')', '4', 'P', 'F'}


Strategy: Throw out the things in PubChem, which contain tokens which aren't in Lense.

In [ ]:
print(alphabet_lense)

In [79]:
shared_alphabet = alphabet_pubchem & alphabet_lense

In [80]:
print(sorted(shared_alphabet))

['#', '(', ')', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', '[N+]', '[N-]', '[NH+]', '[NH3+]', '[O-]', '[S+]']


In [70]:
allowed_alphabet_1 = alphabet_pubchem - alphabet_lense

In [71]:
allowed_alphabet_2 = alphabet_pubchem & alphabet_lense

In [74]:
allowed_alphabet_3 = allowed_alphabet_1.union(allowed_alphabet_2)

In [82]:
print(sorted(allowed_alphabet_3))

['#', '%10', '%11', '%12', '%13', '(', ')', '.', '/', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'B', 'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', '[11CH2]', '[11CH3]', '[11CH]', '[13CH3]', '[13CH]', '[13C]', '[16OH]', '[2H]', '[Ac]', '[Ag]', '[Al]', '[As+3]', '[As]', '[Au+]', '[Au]', '[B-]', '[B]', '[Ba+2]', '[Br-]', '[C+]', '[C-]', '[C@@H]', '[C@@]', '[C@H]', '[C@]', '[CH-]', '[CH2-]', '[CH2]', '[CH3-]', '[CH]', '[C]', '[Cl-]', '[Co+3]', '[Co]', '[Cr+3]', '[Cr]', '[Cs+]', '[Cu+2]', '[Cu+]', '[Er+3]', '[Fe+2]', '[Ga]', '[Gd+3]', '[Ge]', '[H+]', '[HH]', '[Hf]', '[Ho]', '[I-]', '[Ir+3]', '[Ir]', '[K+]', '[Li+]', '[Mg+2]', '[Mg]', '[Mn]', '[Mo+4]', '[Mo]', '[N+]', '[N-]', '[N@@+]', '[NH+]', '[NH-]', '[NH2+]', '[NH2-]', '[NH3+]', '[NH4+]', '[N]', '[Na+]', '[Na]', '[Nd]', '[Ni+2]', '[Ni]', '[O+]', '[O-]', '[OH+]', '[OH-]', '[OH2+]', '[P+]', '[P-]', '[P@@]', '[P@]', '[Pb+2]', '[Pd+2]', '[Pd]', '[Po]', '[Pr]', '[Pt+2]', '[Rh+2]', '[Rh]', '[Ru+]', '[Ru]', '[S+]', '[S-]', '[S@@]', '[S

In [77]:
print(sorted(alphabet_pubchem))

['#', '%10', '%11', '%12', '%13', '(', ')', '.', '/', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'B', 'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', '[11CH2]', '[11CH3]', '[11CH]', '[13CH3]', '[13CH]', '[13C]', '[16OH]', '[2H]', '[Ac]', '[Ag]', '[Al]', '[As+3]', '[As]', '[Au+]', '[Au]', '[B-]', '[B]', '[Ba+2]', '[Br-]', '[C+]', '[C-]', '[C@@H]', '[C@@]', '[C@H]', '[C@]', '[CH-]', '[CH2-]', '[CH2]', '[CH3-]', '[CH]', '[C]', '[Cl-]', '[Co+3]', '[Co]', '[Cr+3]', '[Cr]', '[Cs+]', '[Cu+2]', '[Cu+]', '[Er+3]', '[Fe+2]', '[Ga]', '[Gd+3]', '[Ge]', '[H+]', '[HH]', '[Hf]', '[Ho]', '[I-]', '[Ir+3]', '[Ir]', '[K+]', '[Li+]', '[Mg+2]', '[Mg]', '[Mn]', '[Mo+4]', '[Mo]', '[N+]', '[N-]', '[N@@+]', '[NH+]', '[NH-]', '[NH2+]', '[NH2-]', '[NH3+]', '[NH4+]', '[N]', '[Na+]', '[Na]', '[Nd]', '[Ni+2]', '[Ni]', '[O+]', '[O-]', '[OH+]', '[OH-]', '[OH2+]', '[P+]', '[P-]', '[P@@]', '[P@]', '[Pb+2]', '[Pd+2]', '[Pd]', '[Po]', '[Pr]', '[Pt+2]', '[Rh+2]', '[Rh]', '[Ru+]', '[Ru]', '[S+]', '[S-]', '[S@@]', '[S